# LAB 05 - Data preprocessing

## 04 명목형 변수 인코딩
<범주형 변수 인코딩 개요>
- 문자열 및 범주형 값을 분석 가능한 0 이나 1 등의 숫자로 변환하는 것

### #01.준비작업

In [2]:
from hossam import load_data
from pandas import DataFrame, get_dummies, merge
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import (
    MinMaxScaler, MaxAbsScaler, StandardScaler,
    RobustScaler, Normalizer
)


In [4]:
#샘플 데이터 가져오기
origin = load_data('nursing_grades')
origin

[data] https://data.hossam.kr/data/lab05/nursing_grades.xlsx
[desc] 어느 간호학과 대학원에 지원한 학생들에 대한 합격/불합격 여부를 조사한 가상의 데이터(메타데이터 없음)
[!] Cannot read metadata


,이름,성별,필기점수,학부성적,병원경력,합격여부
접수코드,,,,,,
NRS0001,장은우,남,380,3.61,3,불합격
NRS0002,최지호,남,660,3.67,3,합격
NRS0003,김하준,남,800,4.00,1,합격
NRS0004,임아윤,여,640,3.19,4,합격
NRS0005,강하준,남,520,2.93,4,불합격
...,...,...,...,...,...,...
NRS0396,박지유,여,620,4.00,2,불합격
NRS0397,조하은,여,560,3.04,3,불합격
NRS0398,박하윤,여,460,2.63,2,불합격


In [6]:
#범주형으로 타입 변환
df = origin.astype({'성별':'category','병원경력':'category','합격여부':'category'})

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 400 entries, NRS0001 to NRS0400
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   이름      400 non-null    object  
 1   성별      400 non-null    category
 2   필기점수    400 non-null    int64   
 3   학부성적    400 non-null    float64 
 4   병원경력    400 non-null    category
 5   합격여부    400 non-null    category
dtypes: category(3), float64(1), int64(1), object(1)
memory usage: 14.1+ KB


### #02.Pandas를 사용하는 방법 (일반 통계 모형에 추천)

In [9]:
#더미변수로 변환할 컬럼 이름을 파라미터로 전달
#기존의 컬럼은 제거되고 더미변수로 변경된 컬럼들이 추가
#dtype = int 파라미터를 명시하지 않으면 boolean 으로 생성되므로 int 설정 권장
df1 = get_dummies(df,columns=['성별'],dtype='int')
df1.head()

,이름,필기점수,학부성적,병원경력,합격여부,성별_남,성별_여
접수코드,,,,,,,
NRS0001,장은우,380,3.61,3,불합격,1,0
NRS0002,최지호,660,3.67,3,합격,1,0
NRS0003,김하준,800,4.00,1,합격,1,0
NRS0004,임아윤,640,3.19,4,합격,0,1
NRS0005,강하준,520,2.93,4,불합격,1,0


In [11]:
#일반 통계 모형에선 n-1 개의 더미변수만 생성하기 때문에 drop_first = True 파라미터를 생성한다

df2 = get_dummies(df,columns=['성별'],dtype = 'int',drop_first =True)

df2.head()

,이름,필기점수,학부성적,병원경력,합격여부,성별_여
접수코드,,,,,,
NRS0001,장은우,380,3.61,3,불합격,0
NRS0002,최지호,660,3.67,3,합격,0
NRS0003,김하준,800,4.00,1,합격,0
NRS0004,임아윤,640,3.19,4,합격,1
NRS0005,강하준,520,2.93,4,불합격,0


In [12]:
#2개 이상의 명목형 변수를 한번에 처리할 수도 있따
df3 = get_dummies(df,columns=['성별','병원경력'],dtype = 'int',drop_first=True)
df3.head()

,이름,필기점수,학부성적,합격여부,성별_여,병원경력_2,병원경력_3,병원경력_4
접수코드,,,,,,,,
NRS0001,장은우,380,3.61,불합격,0,0,1,0
NRS0002,최지호,660,3.67,합격,0,0,1,0
NRS0003,김하준,800,4.00,합격,0,0,0,0
NRS0004,임아윤,640,3.19,합격,1,0,0,1
NRS0005,강하준,520,2.93,불합격,0,0,0,1


### #02.Scikit-Learn 을 사용하느 방법 (트리 기반 머신러닝 모형에 추천)
- 1단계  - 방법 (1) 명목형 데이터의 차원 변환
- Sklearn 의 OneHotEncoder 는 2차원 배열이나 DataFrame 객체를 파라미터로 요구하기 떄문에, 인코딩 처리할 명목형 변수의 차원 변환을 수행하거나, 데이터 프레임의 형태를 유지해야한다


- 2단계 - OneHotEncoding 처리
- 3단계 - 인코딩 결과를 데이터 프레임으로 생성
- 4단계 - 원본 데이터 프레임과 병합

In [14]:
#1단계  - 방법 (1) 명목형 데이터의 차원 변환
#방법1) 행 수에 관계 없이 1개의 열을 갖는 2차원 배열로 변환하는 경우

X=df['성별'].values.reshape(-1,1)
X

[['남'], ['남'], ['남'], ['여'], ['남'], ..., ['여'], ['여'], ['여'], ['여'], ['여']]
Length: 400
Categories (2, object): ['남', '여']

In [16]:
#방법 2) 데이터 프레임 형식을 유지하는 변수 추출

X=df[['성별']]

X.head()

,성별
접수코드,
NRS0001,남
NRS0002,남
NRS0003,남
NRS0004,여
NRS0005,남


In [18]:
#2 단계 : OneHotEncoding 처리 (컴퓨터가 문자를 이해할 수 있또록 0과 1로만 된 스위치로 바꿔주는 번역 과정1)000000000

encoder = OneHotEncoder(sparse_output=False , drop=None)   #희소행렬 False 지정 / 첫번째 항목 삭제 여부 None

result = encoder.fit_transform(X)
result

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.

In [20]:
#3단계 : 인코딩 결과를 데이터 프레임으로 생성

#컬럼명 생성
new_cols = encoder.get_feature_names_out(['성별'])
new_cols

array(['성별_남', '성별_여'], dtype=object)

In [22]:
#DataFrame 병합
one_hot_df = DataFrame(result,columns=new_cols, index = df.index)
one_hot_df.head()

,성별_남,성별_여
접수코드,,
NRS0001,1.0,0.0
NRS0002,1.0,0.0
NRS0003,1.0,0.0
NRS0004,0.0,1.0
NRS0005,1.0,0.0


In [23]:
#4단계 : 원본 데이터 프레임과 병합
df_copy = df.copy()
df4= merge(df_copy,one_hot_df,left_index=True,right_index=True)
df4.head()

,이름,성별,필기점수,학부성적,병원경력,합격여부,성별_남,성별_여
접수코드,,,,,,,,
NRS0001,장은우,남,380,3.61,3,불합격,1.0,0.0
NRS0002,최지호,남,660,3.67,3,합격,1.0,0.0
NRS0003,김하준,남,800,4.00,1,합격,1.0,0.0
NRS0004,임아윤,여,640,3.19,4,합격,0.0,1.0
NRS0005,강하준,남,520,2.93,4,불합격,1.0,0.0
